In [ ]:
from model import DEModel
from data import DEData
from torch import nn
from IPython import display

import torch
import plot

In [ ]:
batch_size = 64
ngpu = 0
# Epochs set to '1' for testing
epochs = 1000
learning_rate = 1e-4
beta1 = 0.9

In [ ]:
dataset = DEData('dE_data').trim(-4.9, -4.) + DEData('dE_data_2').trim(-4.9, -4.)

data_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=True,
)

In [ ]:
dE_model = DEModel(len(dataset.start_eta), len(dataset.final_eta))

In [ ]:
optimizer = torch.optim.Adam(dE_model.parameters(), lr=learning_rate, betas=(beta1, 0.999))

In [ ]:
loss_func = nn.MSELoss()

In [ ]:
loss_total = []

for epoch in range(epochs):
    for i, data in enumerate(data_loader):
        actual_batch_size = data[1].shape[0]
        etas = dataset.final_eta
        dE_deta_initial = data[0].reshape(actual_batch_size,1, len(dataset.start_eta))
        dNch_deta_final = data[1].reshape(actual_batch_size, 1, len(dataset.final_eta))

        optimizer.zero_grad()

        output = dE_model(dE_deta_initial.float())

        loss = loss_func(output, dNch_deta_final.float())

        loss.backward()
        optimizer.step()

        #loss_total.append(loss.detach().numpy())

        if i % 128 == 0:
            display.clear_output(wait=True)
            #plot.plot_telemetry( loss_total, output.detach().numpy(), dNch_deta_final, etas)

            print(f"Epoch [{epoch+1}/{epochs}], Batch [{i+1}/{len(dataset) // batch_size}], loss: {loss:.4f}")

In [ ]:
torch.save(dE_model, './dE_model_trim.pt')